In [3]:
import pandas as pd
from sklearn.preprocessing import StandardScaler, MinMaxScaler
import numpy as np

data = pd.read_csv("preprocessed_apartment_rentals_clusters.csv")
# Beispiel-Daten (ersetze X, y durch deine Daten)
features = data.drop(columns=["price", "cityname", "state"]).columns
print(features)
X, y = data.drop(columns=["price", "cityname", "state"]).values, data["price"].values
scaler = StandardScaler()
#scaler = MinMaxScaler
X = scaler.fit_transform(X)

Index(['bathrooms', 'bedrooms', 'has_photo', 'square_feet', 'latitude',
       'longitude', 'time', 'pool', 'gym', 'parking', 'sauna', 'elevator',
       'clubhouse', 'source_Andere source', 'source_GoSection8',
       'source_ListedBuy', 'source_RealRentals', 'source_RentDigs.com',
       'source_RentLingo', 'state_AK', 'state_AL', 'state_AR', 'state_AZ',
       'state_CA', 'state_CO', 'state_CT', 'state_DC', 'state_DE', 'state_FL',
       'state_GA', 'state_HI', 'state_IA', 'state_ID', 'state_IL', 'state_IN',
       'state_KS', 'state_KY', 'state_LA', 'state_MA', 'state_MD', 'state_ME',
       'state_MI', 'state_MN', 'state_MO', 'state_MS', 'state_MT', 'state_NC',
       'state_ND', 'state_NE', 'state_NH', 'state_NJ', 'state_NM', 'state_NV',
       'state_NY', 'state_OH', 'state_OK', 'state_OR', 'state_PA', 'state_RI',
       'state_SC', 'state_SD', 'state_TN', 'state_TX', 'state_UT', 'state_VA',
       'state_VT', 'state_WA', 'state_WI', 'state_WV', 'state_WY', 'cluster_0',
       '

## 🧠 Hyperparameter des MLPRegressor (Neuronales Netz)

Der **MLPRegressor** implementiert ein Feedforward-Neuronales Netz und ist ein komplexes nichtlineares Modell. Die Parameter definieren das Suchgitter (`mlp_params`) für die Optimierung der Architektur und Regulierung in der Nested Cross-Validation.

| Parameter | Beschreibung | Im Code definierte Werte |
| :--- | :--- | :--- |
| **`hidden_layer_sizes`** | Definiert die **Architektur des Netzes**: Die Anzahl der versteckten Schichten (Hidden Layers) und die Anzahl der Neuronen pro Schicht. | `[(10, 10), (50, 50)]` |
| **`activation`** | Die **Aktivierungsfunktion** für die versteckten Schichten. Definiert die Nichtlinearität des Modells. | `["relu"]` |
| **`alpha`** | Der **L2-Regularisierungsterm** (äquivalent zu Ridge Regression). Ein höherer Wert reduziert die Komplexität und hilft, **Overfitting** zu verhindern. | `np.logspace(-5, -2, 4)` |
| **`max_iter`** | Die **maximale Anzahl von Iterationen** (Epochen), die der Optimierer über die Trainingsdaten ausführen soll. | `[500]` |

---

### 💡 Erläuterung der Netzarchitektur



[Image of a feedforward neural network architecture showing input, hidden, and output layers]


* **Netzarchitektur:** `(10, 10)` bedeutet zwei versteckte Schichten mit jeweils 10 Neuronen. `(50, 50)` bedeutet zwei versteckte Schichten mit jeweils 50 Neuronen.
* **`relu` (Rectified Linear Unit):** Sorgt dafür, dass das Netz **Nichtlinearitäten** lernen kann (d.h. komplexere Beziehungen als eine einfache lineare Regression).
* **`alpha` (Regularisierung):** Die Suche über $\text{np.logspace}(-5, -2, 4)$ testet logarithmisch verteilte Werte zwischen $10^{-5}$ und $10^{-2}$. Dies ist entscheidend, um die Modellkomplexität zu kontrollieren.
* **`max_iter`:** Da Neuronale Netze sequenziell lernen, ist ein ausreichender Wert notwendig, um die **Konvergenz** des Optimierers zu gewährleisten.

In [ ]:
from Nested_CV import NestedCVRegressor
from sklearn.neural_network import MLPRegressor

mlp_params = {
    "hidden_layer_sizes": [(10,10), (50, 50)],
    "activation": ["relu"],
    #"alpha": np.logspace(-5, -2, 4),
    "max_iter": [500], # Erhöhen der Iterationen ist oft nötig
    "batch_size": [16, 32]
}

mlp_cv = NestedCVRegressor(MLPRegressor(random_state=42), mlp_params,)
print("Starte MLP Nested CV...")
mlp_cv.run(X, y, output=True)

print("\n--- MLP Ergebnisse ---")
print("MLP Mean R²:", mlp_cv.get_mean_r2())
print("MLP best params:", mlp_cv.get_best_params())
#mlp_cv.plot_scores("MLP Nested CV")

Starte MLP Nested CV...
Fitting 5 folds for each of 6 candidates, totalling 30 fits
[CV] END activation=relu, batch_size=32, hidden_layer_sizes=(10, 10), max_iter=500; total time= 6.4min
[CV] END activation=relu, batch_size=32, hidden_layer_sizes=(10, 10), max_iter=500; total time= 6.8min


c:\Users\User\AppData\Local\Programs\Python\Python311\Lib\site-packages\sklearn\neural_network\_multilayer_perceptron.py:691: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (500) reached and the optimization hasn't converged yet.
  warnings.warn(


[CV] END activation=relu, batch_size=32, hidden_layer_sizes=(10, 10), max_iter=500; total time= 7.8min
[CV] END activation=relu, batch_size=32, hidden_layer_sizes=(10, 10), max_iter=500; total time= 8.0min


c:\Users\User\AppData\Local\Programs\Python\Python311\Lib\site-packages\sklearn\neural_network\_multilayer_perceptron.py:691: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (500) reached and the optimization hasn't converged yet.
  warnings.warn(


[CV] END activation=relu, batch_size=32, hidden_layer_sizes=(10, 10), max_iter=500; total time= 8.2min
[CV] END activation=relu, batch_size=32, hidden_layer_sizes=(50, 50), max_iter=500; total time= 9.6min
[CV] END activation=relu, batch_size=32, hidden_layer_sizes=(50, 50), max_iter=500; total time=10.4min


...

c:\Users\User\AppData\Local\Programs\Python\Python311\Lib\site-packages\sklearn\neural_network\_multilayer_perceptron.py:691: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (500) reached and the optimization hasn't converged yet.
  warnings.warn(


[CV] END activation=relu, batch_size=128, hidden_layer_sizes=(50, 50), max_iter=500; total time= 3.5min


c:\Users\User\AppData\Local\Programs\Python\Python311\Lib\site-packages\sklearn\neural_network\_multilayer_perceptron.py:691: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (500) reached and the optimization hasn't converged yet.
  warnings.warn(


[CV] END activation=relu, batch_size=128, hidden_layer_sizes=(50, 50), max_iter=500; total time= 3.6min
Outer Fold 5/5 | Best Params: {'activation': 'relu', 'batch_size': 32, 'hidden_layer_sizes': (50, 50), 'max_iter': 500} | Fit Time: 683.675s | Outer R²: 0.719 | MSE: 128228.371

--- MLP Ergebnisse ---
MLP Mean R²: 0.7288483935231977
MLP best params: [{'activation': 'relu', 'batch_size': 32, 'hidden_layer_sizes': (50, 50), 'max_iter': 500}, {'activation': 'relu', 'batch_size': 32, 'hidden_layer_sizes': (50, 50), 'max_iter': 500}, {'activation': 'relu', 'batch_size': 32, 'hidden_layer_sizes': (50, 50), 'max_iter': 500}, {'activation': 'relu', 'batch_size': 32, 'hidden_layer_sizes': (50, 50), 'max_iter': 500}, {'activation': 'relu', 'batch_size': 32, 'hidden_layer_sizes': (50, 50), 'max_iter': 500}]


AttributeError: module 'matplotlib' has no attribute 'colors'

In [6]:
print(mlp_cv.get_best_params())

[{'activation': 'relu', 'batch_size': 32, 'hidden_layer_sizes': (50, 50), 'max_iter': 500}, {'activation': 'relu', 'batch_size': 32, 'hidden_layer_sizes': (50, 50), 'max_iter': 500}, {'activation': 'relu', 'batch_size': 32, 'hidden_layer_sizes': (50, 50), 'max_iter': 500}, {'activation': 'relu', 'batch_size': 32, 'hidden_layer_sizes': (50, 50), 'max_iter': 500}, {'activation': 'relu', 'batch_size': 32, 'hidden_layer_sizes': (50, 50), 'max_iter': 500}]


In [5]:
import pandas as pd
number_outer_cv_splits = 5
model_evaluation = pd.DataFrame({"Fold": range(1, number_outer_cv_splits+1)})
model_evaluation['Modell'] = ['Ridge'] * number_outer_cv_splits
model_evaluation['R_2'] = mlp_cv.get_r2_scores()
model_evaluation['MSE'] = mlp_cv.get_mse_scores()
model_evaluation["runtime"] = mlp_cv.get_fit_times()
model_evaluation

,Fold,Modell,R_2,MSE,runtime
0,1,Ridge,0.731695,126947.914399,1020.159841
1,2,Ridge,0.736581,122657.726519,662.722783
2,3,Ridge,0.725016,127229.883336,805.575974
3,4,Ridge,0.732291,126715.135675,957.876599
4,5,Ridge,0.718659,128228.371410,683.675265
